# Proyecto

Gustavo Alvarado. Carnet # 20063401

In [1637]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_recall_fscore_support
from datetime import datetime
from sklearn.externals import joblib

In [1638]:
if tf.__version__.startswith("2."):
    import tensorflow.compat.v1 as tf
    tf.compat.v1.disable_v2_behavior()
    tf.compat.v1.disable_eager_execution()
    print("Enabled compatitility to tf1.x")

Enabled compatitility to tf1.x


In [851]:
titanic_data = pd.read_csv('data_titanic_proyecto.csv')
titanic_data

,PassengerId,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,passenger_class,passenger_sex,passenger_survived
0,1,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,NaN,S,Lower,M,N
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,C,Upper,F,Y
2,3,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Lower,F,Y
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,S,Upper,F,Y
4,5,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,NaN,S,Lower,M,N
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,"Montvila, Rev. Juozas",27.0,0,0,211536,13.0000,NaN,S,Middle,M,N
887,888,"Graham, Miss. Margaret Edith",19.0,0,0,112053,30.0000,B42,S,Upper,F,Y
888,889,"Johnston, Miss. Catherine Helen ""Carrie""",NaN,1,2,W./C. 6607,23.4500,NaN,S,Lower,F,N
889,890,"Behr, Mr. Karl Howell",26.0,0,0,111369,30.0000,C148,C,Upper,M,Y


In [585]:
#Verificando datos faltantes
titanic_data.isna().sum()

PassengerId             0
Name                    0
Age                   177
SibSp                   0
Parch                   0
Ticket                  0
Fare                    0
Cabin                 687
Embarked                2
passenger_class         0
passenger_sex           0
passenger_survived      0
dtype: int64

In [852]:
#Calculando moda para los 2 Embarked faltante 
titanic_data['Embarked'].fillna(titanic_data['Embarked'].mode()[0], inplace = True)
#Calculando media para las edades faltantes
titanic_data['Age'].fillna(titanic_data['Age'].mean(), inplace = True)
#Calculando media para las cuotas faltantes
titanic_data['Fare'].fillna(titanic_data['Fare'].mean(), inplace = True)
#Cabin tiene demasiados faltantes, así que será excluído

In [980]:
titanic_data.isna().sum()

PassengerId             0
Name                    0
Age                     0
SibSp                   0
Parch                   0
Ticket                  0
Fare                    0
Cabin                 687
Embarked                0
passenger_class         0
passenger_sex           0
passenger_survived      0
dtype: int64

## Feature Engineering

**Selección**: Se excluyen los *features* no relevantes: **PassangerId**, **Name**, **Ticket** y **Cabin**.


In [1654]:
selected_data = titanic_data[['Age', 'SibSp', 'Parch', 'Embarked', 'Fare', 'passenger_class', 'passenger_sex', 'passenger_survived']]
selected_data.head()

,Age,SibSp,Parch,Embarked,Fare,passenger_class,passenger_sex,passenger_survived
0,22.0,1,0,S,7.2500,Lower,M,N
1,38.0,1,0,C,71.2833,Upper,F,Y
2,26.0,0,0,S,7.9250,Lower,F,Y
3,35.0,1,0,S,53.1000,Upper,F,Y
4,35.0,0,0,S,8.0500,Lower,M,N


**Transformación**: Transformando datos no numéricos **passenger_class**, **passenger_sex** y **passenger_survived**. 

In [1655]:
transformed_data = selected_data

class_mapping = {'Lower': 1, 'Middle': 2, 'Upper': 3} 
sex_mapping = {'M': 1, 'F': 2}
survived_mapping = {'Y': 1, 'N': 0}
embarked_mapping = {'C': 2, 'Q': 3, 'S': 1}

transformed_data = selected_data.replace({'passenger_class': class_mapping, 
                                          'passenger_sex': sex_mapping, 
                                          'passenger_survived': survived_mapping,
                                          'Embarked': embarked_mapping})
transformed_data.head()

,Age,SibSp,Parch,Embarked,Fare,passenger_class,passenger_sex,passenger_survived
0,22.0,1,0,1,7.2500,1,1,0
1,38.0,1,0,2,71.2833,3,2,1
2,26.0,0,0,1,7.9250,1,2,1
3,35.0,1,0,1,53.1000,3,2,1
4,35.0,0,0,1,8.0500,1,1,0


**Creación**: Creación de nuevas columnas

In [1656]:
complemented_data = transformed_data
#Agregando nueva columna con el total de familia que acompaña al pasajero
complemented_data['total_family'] = complemented_data['SibSp'] + complemented_data['Parch']
#Agregando una nueva columna verificando si viaja solo o acompañado, basado en SibSp (hermanos y cónyuges) y Parch (padres e hijos)
complemented_data.loc[complemented_data['SibSp'] + complemented_data['Parch'] > 0, 'with_family'] = 1
complemented_data.loc[complemented_data['SibSp'] + complemented_data['Parch'] == 0, 'with_family'] = 0

complemented_data

,Age,SibSp,Parch,Embarked,Fare,passenger_class,passenger_sex,passenger_survived,total_family,with_family
0,22.000000,1,0,1,7.2500,1,1,0,1,1.0
1,38.000000,1,0,2,71.2833,3,2,1,1,1.0
2,26.000000,0,0,1,7.9250,1,2,1,0,0.0
3,35.000000,1,0,1,53.1000,3,2,1,1,1.0
4,35.000000,0,0,1,8.0500,1,1,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...
886,27.000000,0,0,1,13.0000,2,1,0,0,0.0
887,19.000000,0,0,1,30.0000,3,2,1,0,0.0
888,29.699118,1,2,1,23.4500,1,2,0,3,1.0
889,26.000000,0,0,2,30.0000,3,1,1,0,0.0


**Agrupando**: Agrupación de *features* por rangos

In [1657]:
grouped_data = complemented_data

In [1658]:
grouped_data['Age'].describe()

count    891.000000
mean      29.699118
std       13.002015
min        0.420000
25%       22.000000
50%       29.699118
75%       35.000000
max       80.000000
Name: Age, dtype: float64

In [1659]:
#Agrupando por edades: 
#1 - Bebés -  0 a 2 años
#2 - Niños - 3 a 12 años
#3 - Adolescentes - 13 a 17 años
#4 - Adultos jóvenes - 18 a 40 años
#5 - Adultos mediana - 41 a 65 años
#6 - Adultos tercera edad - 65 años en adelante
grouped_data['age_range'] = pd.cut(grouped_data['Age'], 
                                   bins=[0, 2, 12, 17, 40, 65, 80], 
                                   labels=[1, 2, 3, 4, 5, 6])
grouped_data

,Age,SibSp,Parch,Embarked,Fare,passenger_class,passenger_sex,passenger_survived,total_family,with_family,age_range
0,22.000000,1,0,1,7.2500,1,1,0,1,1.0,4
1,38.000000,1,0,2,71.2833,3,2,1,1,1.0,4
2,26.000000,0,0,1,7.9250,1,2,1,0,0.0,4
3,35.000000,1,0,1,53.1000,3,2,1,1,1.0,4
4,35.000000,0,0,1,8.0500,1,1,0,0,0.0,4
...,...,...,...,...,...,...,...,...,...,...,...
886,27.000000,0,0,1,13.0000,2,1,0,0,0.0,4
887,19.000000,0,0,1,30.0000,3,2,1,0,0.0,4
888,29.699118,1,2,1,23.4500,1,2,0,3,1.0,4
889,26.000000,0,0,2,30.0000,3,1,1,0,0.0,4


In [1600]:
grouped_data['Fare'].describe()

count    891.000000
mean      32.204208
std       49.693429
min        0.000000
25%        7.910400
50%       14.454200
75%       31.000000
max      512.329200
Name: Fare, dtype: float64

In [1660]:
#Agrupando por tarifa: 
#1 - Tarifa 1 - 0 a 8 
#2 - Tarifa 2 - 8 a 15
#3 - Tarifa 3 - 15 a 31 años
#4 - Tafifa 4 - 31 en adelante
grouped_data['fare_range'] = pd.cut(grouped_data['Fare'], 
                                   bins=[-1, 8, 15, 31, 515], 
                                   labels=[1, 2, 3, 4])
grouped_data

,Age,SibSp,Parch,Embarked,Fare,passenger_class,passenger_sex,passenger_survived,total_family,with_family,age_range,fare_range
0,22.000000,1,0,1,7.2500,1,1,0,1,1.0,4,1
1,38.000000,1,0,2,71.2833,3,2,1,1,1.0,4,4
2,26.000000,0,0,1,7.9250,1,2,1,0,0.0,4,1
3,35.000000,1,0,1,53.1000,3,2,1,1,1.0,4,4
4,35.000000,0,0,1,8.0500,1,1,0,0,0.0,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...
886,27.000000,0,0,1,13.0000,2,1,0,0,0.0,4,2
887,19.000000,0,0,1,30.0000,3,2,1,0,0.0,4,3
888,29.699118,1,2,1,23.4500,1,2,0,3,1.0,4,3
889,26.000000,0,0,2,30.0000,3,1,1,0,0.0,4,3


In [1661]:
final_data = grouped_data

In [1662]:
final_data['fare_range'] = final_data['fare_range'].cat.codes + 1
final_data['age_range'] = final_data['age_range'].cat.codes + 1

In [1663]:
final_data

,Age,SibSp,Parch,Embarked,Fare,passenger_class,passenger_sex,passenger_survived,total_family,with_family,age_range,fare_range
0,22.000000,1,0,1,7.2500,1,1,0,1,1.0,4,1
1,38.000000,1,0,2,71.2833,3,2,1,1,1.0,4,4
2,26.000000,0,0,1,7.9250,1,2,1,0,0.0,4,1
3,35.000000,1,0,1,53.1000,3,2,1,1,1.0,4,4
4,35.000000,0,0,1,8.0500,1,1,0,0,0.0,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...
886,27.000000,0,0,1,13.0000,2,1,0,0,0.0,4,2
887,19.000000,0,0,1,30.0000,3,2,1,0,0.0,4,3
888,29.699118,1,2,1,23.4500,1,2,0,3,1.0,4,3
889,26.000000,0,0,2,30.0000,3,1,1,0,0.0,4,3


## Datos de entrenamiento, validación y de prueba

In [1664]:
x = final_data
y = x.pop('passenger_survived')

In [1665]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.20)
x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size = 0.20)

In [1666]:
x_train

,Age,SibSp,Parch,Embarked,Fare,passenger_class,passenger_sex,total_family,with_family,age_range,fare_range
570,62.000000,0,0,1,10.5000,2,1,0,0.0,5,2
141,22.000000,0,0,1,7.7500,1,2,0,0.0,4,1
699,42.000000,0,0,1,7.6500,1,1,0,0.0,5,1
274,29.699118,0,0,3,7.7500,1,2,0,0.0,4,1
682,20.000000,0,0,1,9.2250,1,1,0,0.0,4,2
...,...,...,...,...,...,...,...,...,...,...,...
482,50.000000,0,0,1,8.0500,1,1,0,0.0,5,2
88,23.000000,3,2,1,263.0000,3,2,5,1.0,4,4
251,29.000000,1,1,1,10.4625,1,2,2,1.0,4,2
439,31.000000,0,0,1,10.5000,2,1,0,0.0,4,2


In [1667]:
y_train

570    1
141    1
699    0
274    1
682    0
      ..
482    0
88     1
251    0
439    0
292    0
Name: passenger_survived, Length: 569, dtype: int64

## Ensemble learning

Nota: Para este proceso de entrenamiento, no se efectuará *bootstraping*, sin embargo, para problemas reales si se recomienda hacerlo. Para efectuar *bootstraping*, se necesita definir el tamaño de la muestra y el número de repeticiones. Se obtiene una muestra con el tamaño definido, utilizando datos aleatorios del dataset, sin importar si se repiten los elementos. El proceso se repite en número de veces definido para obtener las estadísticas requeridas por cada muestra. Esto puede ser efectuado con la función **resample** de **scikit-learn**. 

### Árbol de decisión

In [1646]:
def trainDecisionTree(x_train, y_train, x_test, y_test):
    classifier = DecisionTreeClassifier(random_state = 0)
    model = classifier.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test,y_pred))
    
    #Guardando en bitácora
    conf_string = "DECISION_TREE-" + '-'.join(list(x_train.columns.values))
    accuracy = accuracy_score(y_test, y_pred)
    precision, recall, fscore, _ = precision_recall_fscore_support(y_test, y_pred)
    metrics = [[conf_string, accuracy, precision[0], precision[1], recall[0], recall[1], fscore[0], fscore[1]]]
    log = pd.DataFrame(metrics, 
                       columns=['experiment', 'accuracy', 'precision_0', 'precision_1', 
                                'recall_0', 'recall_1', 'fscore_0', 'fscore_1'])
    log.to_csv('log.csv', index = False, mode = 'a', header = False)
    
    #Guardando el modelo
    joblib.dump(model, './models/' + conf_string + '.pkl')
    return accuracy

In [1647]:
trainDecisionTree(x_train, y_train, x_test, y_test)

[[91 19]
 [24 45]]
              precision    recall  f1-score   support

           0       0.79      0.83      0.81       110
           1       0.70      0.65      0.68        69

    accuracy                           0.76       179
   macro avg       0.75      0.74      0.74       179
weighted avg       0.76      0.76      0.76       179



0.7597765363128491

### SVM

In [1643]:
def trainSVM(x_train, y_train, x_test, y_test):
    svm_classifier = SVC(kernel='linear')
    model = svm_classifier.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    
    #Guardando en bitácora
    conf_string = "SVM-" + '-'.join(list(x_train.columns.values))
    accuracy = accuracy_score(y_test, y_pred)
    precision, recall, fscore, _ = precision_recall_fscore_support(y_test, y_pred)
    metrics = [[conf_string, accuracy, precision[0], precision[1], recall[0], recall[1], fscore[0], fscore[1]]]
    log = pd.DataFrame(metrics, 
                       columns=['experiment', 'accuracy', 'precision_0', 'precision_1', 
                                'recall_0', 'recall_1', 'fscore_0', 'fscore_1'])
    log.to_csv('log.csv', index = False, mode = 'a', header = False)
    
    #Guardando el modelo
    joblib.dump(model, './models/' + conf_string + '.pkl')
    return model

In [1645]:
trainSVM(x_train, y_train, x_test, y_test)

[[95 15]
 [23 46]]
              precision    recall  f1-score   support

           0       0.81      0.86      0.83       110
           1       0.75      0.67      0.71        69

    accuracy                           0.79       179
   macro avg       0.78      0.77      0.77       179
weighted avg       0.79      0.79      0.78       179



SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

### Naive Bayes

In [1613]:
def P_x_given_y(x, mean_y, variance_y):
    P = 1/(np.sqrt(2*np.pi*variance_y)) * np.exp((-(x-mean_y)**2)/(2*variance_y))
    return P

In [1650]:
def trainNaiveBayes(x_train, y_train, x_test, y_test):
    data = x_train
    col_name = y_train.to_frame().columns[0]
    data[col_name] = y_train
    
    #Calculando priors
    ones = data[col_name][data[col_name] == 1].count()
    zeros = data[col_name][data[col_name] == 0].count()
    n = len(data)
    P_ones = ones / n
    P_zeros = zeros / n
    
    #Calculando likelihood
    data_means = data.groupby(col_name).mean()
    data_var = data.groupby(col_name).var()
    
    #Predicción
    y_pred = y_test.to_frame()
    y_pred['P_one'] = P_ones
    y_pred['P_zero'] = P_zeros
    y_pred['prediction'] = -1
    
    for i in range(0, len(x_test)):
        for j in range(len(x_test.columns) - 1):
            row_name = x_test.index[i]
            column_name = x_test.columns[j]
            x_one_mean = data_means.loc[1, column_name]
            x_one_var = data_var.loc[1, column_name]
            x_zero_mean = data_means.loc[0, column_name]
            x_zero_var = data_var.loc[0, column_name]
            
            y_pred.loc[row_name, 'P_one'] *= P_x_given_y(x_test.loc[row_name, column_name], x_one_mean, x_one_var)
            y_pred.loc[row_name, 'P_zero'] *= P_x_given_y(x_test.loc[row_name, column_name], x_zero_mean, x_zero_var)
            
            if y_pred.loc[row_name, 'P_one'] > y_pred.loc[row_name, 'P_zero']:
                y_pred.loc[row_name, 'prediction'] = 1
            else:
                y_pred.loc[row_name, 'prediction'] = 0
    

    #Guardando en bitácora
    conf_string = "NAIVE_BAYES-" + '-'.join(list(x_train.columns.values))
    accuracy = accuracy_score(y_test, y_pred['prediction'])
    precision, recall, fscore, _ = precision_recall_fscore_support(y_test, y_pred['prediction'])
    metrics = [[conf_string, accuracy, precision[0], precision[1], recall[0], recall[1], fscore[0], fscore[1]]]
    log = pd.DataFrame(metrics, 
                       columns=['experiment', 'accuracy', 'precision_0', 'precision_1', 
                                'recall_0', 'recall_1', 'fscore_0', 'fscore_1'])
    log.to_csv('log.csv', index = False, mode = 'a', header = False)
    
    print('Accuracy:', accuracy)
    
    #Guardando el modelo
    y_pred['prediction'].to_numpy().dump('./models/' + conf_string + '.npy')
    return accuracy

In [1651]:
trainNaiveBayes(x_train, y_train, x_test, y_test)

Accuracy: 0.8044692737430168


0.8044692737430168

### Regresión logística

In [1652]:
def trainLogisticRegression(x_train, y_train, x_test, y_test, EPOCHS, show_each, learning_rate, lambda_value):
    n_train = len(x_train)
    n_test = len(x_test)
    m = len(x_test.columns)
    column_names = list(x_train.columns.values)
    x_train = x_train.values
    x_train = np.c_[np.ones((n_train)), x_train]
    x_test = x_test.values
    x_test = np.c_[np.ones((n_test)), x_test]
    y_train = np.array([y_train]).T
    y_test = np.array([y_test]).T

    tf.reset_default_graph()
    
    weights = tf.get_variable("weights", dtype=tf.float32, shape=[m + 1, 1], initializer=tf.ones_initializer())
    
    tensor_x_train = tf.placeholder(tf.float32,[n_train, m + 1],"tensor_x_train")
    tensor_y_train = tf.placeholder(tf.float32,[n_train, 1],"tensor_y_train")
    tensor_x_test = tf.placeholder(tf.float32,[n_test, m + 1],"tensor_x_test")
    tensor_y_test = tf.placeholder(tf.float32,[n_test, 1],"tensor_y_test")
    
    #Función sigmoid
    logits = tf.matmul(tensor_x_train, weights)
    y_pred = tf.round(tf.nn.sigmoid(logits))
    loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = logits, labels = tensor_y_train))
    
    #Aplicando regularización
    regularizer = tf.nn.l2_loss(weights)
    cost = tf.reduce_mean(loss + lambda_value * regularizer)
    
    #Gradient descent
    gradient = tf.gradients(cost, [weights])[0]
    weights_update = tf.assign(weights, weights - learning_rate * gradient)
    
    init = tf.global_variables_initializer() 
    
    with tf.Session() as session:
        session.run(init)
        feed_dict = {tensor_x_train: x_train, tensor_y_train: y_train, tensor_x_test: x_test, tensor_y_test: y_test}
            
        for i in range(EPOCHS + 1):    
            cost_value = session.run(cost, feed_dict=feed_dict)
            session.run(weights_update, feed_dict=feed_dict)
            
            comparer = tf.equal(y_pred, tensor_y_train)
            hits = tf.reduce_sum(tf.cast(comparer, tf.float32))
            hits_value = session.run(hits, feed_dict=feed_dict)
            accuracy_value = round(hits_value / n_train, 2)
            if (i % show_each == 0):
                print("Iteration", i, ": Cost =", cost_value, ". Accuracy =", accuracy_value)
       
        w = session.run(weights, feed_dict=feed_dict)
        
        #Probando modelo con datos de prueba
        logits_test = tf.matmul(tensor_x_test, weights)
        y_pred_test = tf.round(tf.nn.sigmoid(logits_test))
        y_pred_test_value = session.run(y_pred_test, feed_dict=feed_dict)
        
    #Guardando en bitácora
    conf_string = "LOGISTIC_REGRESSION-lr=" + str(learning_rate) + "-reg=" + str(lambda_value) + '-'.join(column_names)
    accuracy = accuracy_score(y_test, y_pred_test_value)
    precision, recall, fscore, _ = precision_recall_fscore_support(y_test, y_pred_test_value)
    metrics = [[conf_string, accuracy, precision[0], precision[1], recall[0], recall[1], fscore[0], fscore[1]]]
    log = pd.DataFrame(metrics, 
                       columns=['experiment', 'accuracy', 'precision_0', 'precision_1', 
                                'recall_0', 'recall_1', 'fscore_0', 'fscore_1'])
    log.to_csv('log.csv', index = False, mode = 'a', header = False)
    
    print('Accuracy:', accuracy)
    
    #Guardando modelo 
    w.dump('./models/' + conf_string +'.npy')
    return accuracy

In [1668]:
trainLogisticRegression(x_train, y_train, x_test, y_test, 1000, 100, 0.01, 0.000000001)

Iteration 0 : Cost = 40.209007 . Accuracy = 0.38
Iteration 100 : Cost = 1.7719069 . Accuracy = 0.59
Iteration 200 : Cost = 1.461233 . Accuracy = 0.61
Iteration 300 : Cost = 1.2184668 . Accuracy = 0.59
Iteration 400 : Cost = 1.0973495 . Accuracy = 0.58
Iteration 500 : Cost = 0.9193613 . Accuracy = 0.6
Iteration 600 : Cost = 1.1618419 . Accuracy = 0.67
Iteration 700 : Cost = 1.2653083 . Accuracy = 0.44
Iteration 800 : Cost = 0.95553595 . Accuracy = 0.4
Iteration 900 : Cost = 0.7704818 . Accuracy = 0.42
Iteration 1000 : Cost = 1.0443304 . Accuracy = 0.75
Accuracy: 0.7597765363128491


0.7597765363128491

## K-Folds Cross Validation

Es un método estadístico utilizado para estimar la capacidad de modelos de *machine learning*. Utiliza un único parámetro **k** que determinará el número de grupos en la que una muestra se dividirá. El procedimiento empieza dividiendo el *data set* de entrada aleatoramiente en **k** grupos. Para cada grupo, el mismo se toma como data de prueba, mientras que el resto se toma como data de entrenamiento. Por cada grupo existirá un proceso de entrenamiento y se registran sus resultados. Al final, se sumarizan todos los resultados para tener un dato promediado. Este proceso se puede realizar por medio de la función **KFold** de **scikit-learn**.

## Evaluación final

In [ ]:
def ensembleLearning(x_train, y_train, x_test, y_test):
    accuracy1 = 
    

# Conclusiones y recomendaciones

# Deployment